# Emmenhtal
> Peeling the layers of this polyglot loader
- toc: true 
- badges: true
- categories: [Emmenhtal,polygot,loader]

## Overview

This is a polyglot loader which masqurades as a legit PE file but is really a way to deliver an HTA script. The loader is often launched via a malicious `.lnk` file and has been observed downloading  CryptBot or Lumma stealer.

### References

- [Orange Cyberdefense](https://www.orangecyberdefense.com/global/blog/cert-news/emmenhtal-a-little-known-loader-distributing-commodity-infostealers-worldwide)  Emmenhtal: a little-known loader distributing commodity infostealers worldwider
- [Mandiant](https://cloud.google.com/blog/topics/threat-intelligence/peaklight-decoding-stealthy-memory-only-malware/)  PEAKLIGHT: Decoding the Stealthy Memory-Only Malware
- [Cisco Talos](https://blog.talosintelligence.com/suspected-coralraider-continues-to-expand-victimology-using-three-information-stealers/)  Suspected CoralRaider continues to expand victimology using three information stealers


### Sample
- `dd52a6b3e9d1f368ed000d5a506331ce5b3f194512f9d075b494510ae1583a1f` [UnpacMe](https://www.unpac.me/results/d3aaef6f-cb0b-46c1-b3db-d8cc552bf067)

## Analysis

According to [Orange Cyberdefense](https://www.orangecyberdefense.com/global/blog/cert-news/emmenhtal-a-little-known-loader-distributing-commodity-infostealers-worldwide) the delivery chain involves a .lnk file that is used to launch a plyglot EXE/HTA file with powershell and mshta. This is a similar technique to one used by Zloader in 2022 according to [CheckPoint](https://research.checkpoint.com/2022/can-you-trust-a-files-digital-signature-new-zloader-campaign-exploits-microsofts-signature-verification-putting-users-at-risk/).

The polyglot is constructed by compbining a beign PE file (for example `notepad.exe`) with an HTA script. The HTA script is split into parts (header tag, script, end header tag) and the benign PE file is pasted between each part. The resulting file appears to be a PE file, and does contain multiple copies of the benign PE, however when executed with `mshta` only the script components are read by the HTA parser and the file becomes an HTA document. This launches a a chain of obfuscated script stages which eventually download and run the payload.

![](https://www.orangecyberdefense.com/fileadmin/_processed_/d/c/csm_Image1_d9f5086570.webp)



### Polyglot Analysis

Stage1 extracts the HTA script from the polyglot. The script is heavily obfuscated.

[Extracted Stage1](https://pastebin.com/nwmxp0j9)

In [ ]:
file_data = open('/tmp/dd52a6b3e9d1f368ed000d5a506331ce5b3f194512f9d075b494510ae1583a1f', 'rb').read()

script = b''
recording = False
for i in range(len(file_data)):
    # Check for <script> token
    if file_data[i:i+8] == b'<script>':
        # Start recording the script
        recording = True
    # Check if we are recording and we find the </script> token
    if recording and file_data[i:i+9] == b'</script>':
        script += file_data[i:i+9]
        # Stop recording the script
        recording = False
    if recording:
        script += file_data[i:i+1]
    i += 1

print(script)

#### Stage2

Stage1 can be decrupted using a simple substitution algorithm to build character array which is then evaluated revealing Stage 2.

[Extracted stage2](https://pastebin.com/iK1BEu2T)


#### Stage3

Stage 2 can be deobfuscated with a similar character array only this time subtracting a fixed value from each characte.

[Extracted stage3](https://pastebin.com/0EzgNB0T)


#### Stage4

Stage 4 is a powershell script that uses AES CBC with a null IV and a hard coded key to decrypt stage 5

[Extracted stage4](https://pastebin.com/vv3BfKTC)

#### Stage 5

Stage 5 is also a powershell script used to download and execute the final payload.

[Extracted deobfuscated stage5](https://pastebin.com/n97XESRM)


In [1]:
bytes([104 , 116 , 116 , 112 , 115 , 58, 47, 47, 115 , 104 , 111 , 119 , 45, 112 , 100 , 102 , 45, 100 , 111 , 99, 117 , 109 , 101 , 110 , 116 , 46, 99, 111 , 109 , 47, 114 , 101 , 110 , 116 , 97, 112 , 105 , 47, 117 , 112 , 100 , 97, 116 , 101 , 46, 98, 105 , 110])

b'https://show-pdf-document.com/rentapi/update.bin'